# Домашнее задание: реализовать GPT-модель с Mixture of Experts слоями.

## Домашнее задание

- Расширьте данный пример, добавив реализацию RMSNorm и rotary embeddings. **(4 балла)**
- Проведите эксперимент по изменению числа экспертов и размера модели. **(4 балла)**
- Опишите, как scaling laws влияют на производительность модели. **(2 балла)**


## Обзор Mixture of Experts (MoE)

Mixture of Experts (MoE) – это подход, позволяющий масштабировать модели путём распределения вычислительных задач между несколькими "экспертами". Для каждого входа вычисляется распределение вероятностей по экспертам с помощью "гейтинговой" сети, и итоговый выход получается как взвешенная сумма выходов экспертов.

### Задачи метода MoE:
1. Увеличение параметров модели без линейного роста вычислительной нагрузки.
2. Обучение специализированных подсетей для различных аспектов данных.

В следующих ячейках представлен упрощённый пример реализации MoE-слоя и его интеграции в трансформер-блок.

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Реализация MoE-слоя

В этой части мы создадим упрощённый класс MoE, где каждый эксперт – это простая линейная трансформация, а гейтинговая сеть определяет веса для каждого эксперта.

In [26]:
# Пример шаблонной реализации MoE (требует доработки)


class MixtureOfExperts(nn.Module):
    def __init__(self, input_dim, output_dim, num_experts=4):
        super(MixtureOfExperts, self).__init__()
        self.num_experts = num_experts

        # создаь список экспертов (каждый эксперт - линейное преобразование)
        self.experts = nn.ModuleList(
            [nn.Linear(input_dim, output_dim) for _ in range(num_experts)]
        )

        # Гейтинговая сеть для определения весов
        self.gate = nn.Linear(input_dim, num_experts)

    def forward(self, x):
        # вычислите gate_scores, gate_probs
        # примените экспертов к входу x и объедините результаты с использованием весов
        # подсказка: используйте torch.stack и torch.sum
        # например, gate_scores = self.gate(x)

        gate_scores = self.gate(x)
        gate_probs = F.softmax(gate_scores, dim=-1)

        # Применяем каждого эксперта
        expert_outputs = torch.stack([expert(x) for expert in self.experts], dim=-1)
        # Расширьте размеры gate_probs для совместимости
        gate_probs = gate_probs.unsqueeze(2)

        # Взвешенная сумма выходов экспертов
        output = torch.sum(expert_outputs * gate_probs, dim=-1)
        return output

#### Проверьте работу MoE-слоя на тестовом входе (при необходимости)

## Объяснение реализации MoE-слоя

- **self.experts:** Список линейных слоёв, каждый из которых является "экспертом".
- **self.gate:** Линейный слой, выдающий веса для каждого эксперта.
- **forward:** Вычисляется softmax по выходу гейта, затем каждое линейное преобразование применяется к входу и комбинируется с помощью весов.


In [21]:
# класс TransformerBlockMoE (требует доработки)


class TransformerBlockMoE(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_experts=4):
        super(TransformerBlockMoE, self).__init__()
        # многоголовое внимание
        self.attn = nn.MultiheadAttention(
            embed_dim=d_model, num_heads=num_heads, batch_first=True
        )
        # слои нормализации
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        # MoE слой вместо стандартного feed-forward слоя
        self.moe = MixtureOfExperts(d_model, d_ff, num_experts=num_experts)
        # приводим к исходной размерности
        self.fc = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # шаг 1: Многоголовое внимание
        attn_output, _ = self.attn(x, x, x)
        x = self.norm1(x + attn_output)

        # шаг 2: Применение MoE слоя с активацией ReLU
        moe_output = self.moe(x)
        ff_output = self.fc(F.relu(moe_output))
        x = self.norm2(x + ff_output)
        return x

## Объяснение трансформер-блока с MoE

- **Многоголовое внимание:** Стандартное self-attention.
- **norm1 и norm2:** Слои нормализации для стабилизации обучения.
- **MoE-слой:** Заменяет обычный feed-forward слой.
- **fc:** Линейное преобразование для приведения размерности обратно к d_model.


## Тестирование трансформер-блока с MoE

В следующей ячейке создадим тестовый пример:
- Сгенерируем случайный вход (например, эмбеддинги токенов)
- Пропустим вход через блок и выведем форму выходного тензора.


In [22]:
# Тестовый пример использования TransformerBlockMoE

# Задайте параметры модели
batch_size = 2
seq_len = 10
d_model = 32
num_heads = 4
d_ff = 64

# Сгенерируйте случайный вход
x = torch.randn(batch_size, seq_len, d_model)

# Инициализируйте блок трансформера с MoE
transformer_block = TransformerBlockMoE(d_model, num_heads, d_ff, num_experts=4)

# Пропустите вход через блок
output = transformer_block(x)

# Выведите формы входного и выходного тензоров
print("Форма входного тензора:", x.shape)
print("Форма выходного тензора:", output.shape)

Форма входного тензора: torch.Size([2, 10, 32])
Форма выходного тензора: torch.Size([2, 10, 32])


## Дополнительные темы для изучения

- **RMSNorm:** Альтернатива стандартной нормализации для улучшения обучения.
- **Rotary embeddings:** Улучшают представление позиционной информации.
- **Grouped Query Attention:** Модификация механизма внимания для повышения эффективности.

*Попробуйте самостоятельно интегрировать эти элементы в модель.*
